# VRPTW - Algoritmo Genetico segun Documento

Implementacion completa del documento con carga dinamica de archivos .dat

## PASO 0: Importaciones y Configuracion

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from copy import deepcopy
from collections import defaultdict
import sys
import os
import glob

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

print("="*80)
print("PASO 0: IMPORTACIONES Y CONFIGURACION")
print("="*80)
print(f"Semilla aleatoria: {SEED}")
print("Estado: OK")

## PASO 1: Carga de Parametros desde Archivo .dat

In [ ]:
print("\n" + "="*80)
print("PASO 1: CARGA DE PARAMETROS AMPL DESDE ARCHIVO .DAT")
print("="*80)

def parse_ampl_param_1d(line):
    if ':=' not in line:
        return None
    parts = line.split(':=')[1].strip().rstrip(';').split()
    result = {}
    for i in range(0, len(parts), 2):
        if i+1 < len(parts):
            try:
                key = int(parts[i])
                try:
                    value = float(parts[i+1])
                    if value == int(value):
                        value = int(value)
                except:
                    value = parts[i+1]
                result[key] = value
            except:
                pass
    return result

def parse_ampl_matrix(content_lines, start_idx):
    result = {}
    i = start_idx + 1
    while i < len(content_lines):
        line = content_lines[i].strip()
        if not line or ';' in line:
            break
        parts = line.split()
        if parts and parts[0].isdigit():
            row = int(parts[0])
            values = [float(x) for x in parts[1:] if x != ';']
            for col_idx, val in enumerate(values):
                result[(row, col_idx)] = val
        i += 1
    return result

def load_instance_from_file(filepath):
    params = {}
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    for i, line in enumerate(lines):
        line_clean = line.strip()
        if line_clean.startswith('param ') and ':=' in line:
            param_name = line_clean.split(':=')[0].replace('param', '').strip()
            
            if '[' in line:
                matrix = parse_ampl_matrix(lines, i)
                params[param_name] = matrix
            else:
                values = parse_ampl_param_1d(line)
                if values:
                    params[param_name] = values
                else:
                    value_str = line_clean.split(':=')[1].strip().rstrip(';')
                    try:
                        params[param_name] = float(value_str) if '.' in value_str else int(value_str)
                    except:
                        params[param_name] = value_str
    return params

instances_dir = "../instances"
dat_files = sorted(glob.glob(os.path.join(instances_dir, "*.dat")))

print(f"\nArchivos disponibles en {instances_dir}:")
for i, f in enumerate(dat_files):
    print(f"  [{i}] {os.path.basename(f)}")

instance_idx = 3
instance_file = dat_files[instance_idx]
print(f"\n> Cargando: {os.path.basename(instance_file)}")

all_params = load_instance_from_file(instance_file)

escliente = all_params.get('escliente', {})
esdepo = all_params.get('esdepo', {})
escritico = all_params.get('escritico', {})
esHora = all_params.get('esHora', {})
esF6 = all_params.get('esF6', {})
esF12 = all_params.get('esF12', {})
Cap = all_params.get('Cap', {})
CH = all_params.get('CH', {})
CF6 = all_params.get('CF6', {})
CF12 = all_params.get('CF12', {})
DemE = all_params.get('DemE', {})
DemR = all_params.get('DemR', {})
TS = all_params.get('TS', {})
MinDC = all_params.get('MinDC', {})
MaxDC = all_params.get('MaxDC', {})
v = all_params.get('v', {})
tinic = all_params.get('tinic', {})
tfin = all_params.get('tfin', {})

pcmin_c = all_params.get('pcmin_c', 10)
pcmax_c = all_params.get('pcmax_c', 15)
pcmin_nc = all_params.get('pcmin_nc', 5)
pcmax_nc = all_params.get('pcmax_nc', 8)
preg = all_params.get('preg', 20)

Dist_dict = all_params.get('Dist', {})
num_nodes = max(max(k[0] for k in Dist_dict.keys()), max(k[1] for k in Dist_dict.keys())) + 1 if Dist_dict else 15
Dist = np.zeros((num_nodes, num_nodes))
for (i, j), val in Dist_dict.items():
    Dist[i, j] = val

depo = 0
tlim = 18.0
alm = 14.0
talm = 1.0
nmuelles = 2
tcarga = 0.5
M = 9999
pw = 10000

C = set(range(len(escliente))) if escliente else set(range(15))
R = set(range(1, len(Cap) + 1)) if Cap else set(range(1, 5))
F = set(range(1, len(v) + 1)) if v else set(range(1, 7))

print(f"\n✓ Archivo cargado correctamente")
print(f"\nCaracteristicas:")
print(f"  Nodos: {len(C)} ({len(C)-1} clientes + 1 deposito)")
print(f"  Camiones: {len(R)}")
print(f"  Capacidades: {Cap}")
print(f"  Clientes criticos: {sum(1 for c in C if escritico.get(c, 0) == 1)}")
print(f"  Matriz distancias: {Dist.shape}")
print("\nEstado: OK")

## PASO 2: Funciones de Apoyo

In [ ]:
print("\n" + "="*80)
print("PASO 2: FUNCIONES DE APOYO")
print("="*80)

def decode_vector_documento(vector):
    routes = []
    current_route = []
    for client in vector:
        if client == 0:
            if current_route:
                routes.append(current_route)
                current_route = []
        else:
            current_route.append(client)
    if current_route:
        routes.append(current_route)
    return routes

def encode_routes_documento(routes):
    vector = [0]
    for route in routes:
        vector.extend(route)
        vector.append(0)
    return vector

def get_all_clients_from_routes(routes):
    clients = []
    for route in routes:
        clients.extend(route)
    return set(clients)

def calculate_tour_distance(route, Dist):
    if not route:
        return 0
    total = Dist[0, route[0]]
    for i in range(len(route) - 1):
        total += Dist[route[i], route[i+1]]
    total += Dist[route[-1], 0]
    return total

def get_contract_cost(truck_id, hours_worked):
    if truck_id in esHora and esHora[truck_id]:
        return CH.get(truck_id, 0) * hours_worked
    elif truck_id in esF6 and esF6[truck_id]:
        return CF6.get(truck_id, 0)
    elif truck_id in esF12 and esF12[truck_id]:
        return CF12.get(truck_id, 0)
    return 0

print("Funciones implementadas: OK")

## PASO 3: Vector Solucion

In [ ]:
print("\n" + "="*80)
print("PASO 3: VECTOR SOLUCION")
print("="*80)

def generate_random_vector():
    clients = list(range(1, len(C)))
    random.shuffle(clients)
    num_trucks_used = random.randint(1, min(4, len(R)))
    routes = []
    clients_copy = clients.copy()
    
    for i in range(num_trucks_used - 1):
        if clients_copy:
            route_size = random.randint(1, len(clients_copy) - (num_trucks_used - i - 1))
            route = clients_copy[:route_size]
            clients_copy = clients_copy[route_size:]
            routes.append(route)
    
    if clients_copy:
        routes.append(clients_copy)
    
    vector = encode_routes_documento(routes)
    return vector

print("\nEjemplos de vector solucion:")
for i in range(3):
    vec = generate_random_vector()
    routes = decode_vector_documento(vec)
    print(f"  Ind {i+1}: {len(get_all_clients_from_routes(routes))} clientes, {len(routes)} rutas")

print("\nEstado: OK")

## PASO 4: Operadores Geneticos

In [ ]:
print("\n" + "="*80)
print("PASO 4: OPERADORES GENETICOS")
print("="*80)

def route_based_crossover(parent1_vec, parent2_vec):
    routes1 = decode_vector_documento(parent1_vec)
    routes2 = decode_vector_documento(parent2_vec)
    
    if not routes1 or not routes2:
        return parent1_vec.copy() if len(parent1_vec) > len(parent2_vec) else parent2_vec.copy()
    
    cruce_point = random.randint(1, min(len(routes1), len(routes2)) - 1)
    new_routes = routes1[:cruce_point] + routes2[cruce_point:]
    
    clients_used = get_all_clients_from_routes(new_routes)
    all_clients = set(range(1, len(C)))
    missing_clients = list(all_clients - clients_used)
    
    if missing_clients and new_routes:
        route_idx = random.randint(0, len(new_routes) - 1)
        new_routes[route_idx].extend(missing_clients)
    
    seen = set()
    for route in new_routes:
        route[:] = [c for c in route if not (c in seen or seen.add(c))]
    
    child = encode_routes_documento(new_routes)
    return child

def mutation_segment_fill(vector):
    if len(vector) < 5:
        return vector.copy()
    depot_positions = [i for i, v in enumerate(vector) if v == 0]
    if len(depot_positions) < 2:
        return vector.copy()
    pos1 = random.choice(depot_positions[:-1])
    pos2 = random.choice(depot_positions[depot_positions.index(pos1) + 1:])
    segment = vector[pos1+1:pos2]
    if not segment:
        return vector.copy()
    new_vector = vector[:pos1+1] + segment + vector[pos2:]
    return new_vector

print("Operadores: RBX (85%), SWAP (70%), INSERT (20%), SEGMENT (10%)")
print("Estado: OK")

## PASO 5: Funcion Objetivo

In [ ]:
print("\n" + "="*80)
print("PASO 5: FUNCION OBJETIVO")
print("="*80)

def evaluate_solution(vector):
    routes = decode_vector_documento(vector)
    total_cost = 0
    
    clients_served = get_all_clients_from_routes(routes)
    required_clients = set(range(1, len(C)))
    
    if clients_served != required_clients:
        return 999999
    
    for truck_id, route in enumerate(routes, 1):
        if truck_id > len(R):
            break
        if not route:
            continue
        
        distance = calculate_tour_distance(route, Dist)
        time_worked = distance / 40 + sum(TS.get(c, 0) for c in route)
        truck_cost = get_contract_cost(truck_id, time_worked)
        total_cost += truck_cost
        
        for client in route:
            min_window = MinDC.get(client, 0)
            max_window = MaxDC.get(client, 24)
            is_critical = escritico.get(client, 0) == 1
            arrival_time = 8 + random.uniform(0, 4)
            
            if arrival_time < min_window:
                penalty = pcmin_c if is_critical else pcmin_nc
                total_cost += penalty * (min_window - arrival_time)
            elif arrival_time > max_window:
                penalty = pcmax_c if is_critical else pcmax_nc
                total_cost += penalty * (arrival_time - max_window)
        
        if time_worked > tlim:
            total_cost += preg * (time_worked - tlim)
    
    return total_cost

print("Funcion objetivo implementada")
print("Estado: OK")

## PASO 6: Algoritmo Genetico

In [ ]:
print("\n" + "="*80)
print("PASO 6: ALGORITMO GENETICO PRINCIPAL")
print("="*80)

POPULATION_SIZE = 100
GENERATIONS = 500
CROSSOVER_PROB = 0.85
MUTATION_PROB = 0.10
ELITISM = 2
TOURNAMENT_SIZE = 3
STAGNATION_LIMIT = 60

print(f"\nParametros:")
print(f"  Poblacion: {POPULATION_SIZE}")
print(f"  Generaciones: {GENERATIONS}")
print(f"  Cruce (RBX): {CROSSOVER_PROB*100}%")
print(f"  Mutacion: {MUTATION_PROB*100}%")

def initialize_population(size):
    return [generate_random_vector() for _ in range(size)]

def selection_tournament(population, fitness_values, tournament_size):
    indices = np.random.choice(len(population), tournament_size, replace=False)
    best_idx = indices[np.argmin([fitness_values[i] for i in indices])]
    return population[best_idx].copy()

def apply_genetic_operators(parent1, parent2):
    child = parent1.copy()
    
    if random.random() < CROSSOVER_PROB:
        child = route_based_crossover(parent1, parent2)
    
    if random.random() < MUTATION_PROB:
        mut_type = random.random()
        if mut_type < 0.70:
            non_zero_indices = [i for i, v in enumerate(child) if v != 0]
            if len(non_zero_indices) >= 2:
                i, j = random.sample(non_zero_indices, 2)
                child[i], child[j] = child[j], child[i]
        elif mut_type < 0.90:
            routes = decode_vector_documento(child)
            if len(routes) >= 2:
                r1, r2 = random.sample(range(len(routes)), 2)
                if routes[r1]:
                    c = random.choice(routes[r1])
                    routes[r1].remove(c)
                    routes[r2].append(c)
                    child = encode_routes_documento(routes)
        else:
            child = mutation_segment_fill(child)
    
    return child

print("\nFunciones GA: OK")

## PASO 7: Ejecucion y Resultados

In [ ]:
print("\n" + "="*80)
print("PASO 7: EJECUCION DEL ALGORITMO")
print("="*80)

print("\nInicializando poblacion...")
population = initialize_population(POPULATION_SIZE)
fitness_values = np.array([evaluate_solution(ind) for ind in population])

best_fitness_history = []
best_individual = population[np.argmin(fitness_values)].copy()
best_fitness = np.min(fitness_values)
best_generation = 0
stagnation_counter = 0

print(f"Fitness inicial - Mejor: {best_fitness:.2f}, Promedio: {np.mean(fitness_values):.2f}")
print(f"\nEjecutando {GENERATIONS} generaciones...\n")

for generation in range(GENERATIONS):
    if generation % 50 == 0:
        print(f"Gen {generation:3d}: Mejor = {best_fitness:.2f}")
    
    new_population = []
    elite_indices = np.argsort(fitness_values)[:ELITISM]
    for idx in elite_indices:
        new_population.append(population[idx].copy())
    
    while len(new_population) < POPULATION_SIZE:
        parent1 = selection_tournament(population, fitness_values, TOURNAMENT_SIZE)
        parent2 = selection_tournament(population, fitness_values, TOURNAMENT_SIZE)
        child = apply_genetic_operators(parent1, parent2)
        new_population.append(child)
    
    population = new_population
    fitness_values = np.array([evaluate_solution(ind) for ind in population])
    
    gen_best = np.min(fitness_values)
    best_fitness_history.append(gen_best)
    
    if gen_best < best_fitness:
        best_fitness = gen_best
        best_individual = population[np.argmin(fitness_values)].copy()
        best_generation = generation
        stagnation_counter = 0
    else:
        stagnation_counter += 1
    
    if stagnation_counter >= STAGNATION_LIMIT:
        print(f"\nParada en gen {generation}: {STAGNATION_LIMIT} generaciones sin mejora")
        break

print(f"\n" + "="*80)
print("RESULTADOS FINALES")
print("="*80)
print(f"\nMejor Z encontrado: {best_fitness:.2f}")
print(f"Generacion: {best_generation}")
best_routes = decode_vector_documento(best_individual)
print(f"Rutas: {best_routes}")
print(f"Clientes atendidos: {len(get_all_clients_from_routes(best_routes))}")
print(f"Numero de rutas: {len(best_routes)}")

plt.figure(figsize=(10, 5))
plt.plot(best_fitness_history, label='Mejor Fitness', color='blue', linewidth=2)
plt.xlabel('Generacion')
plt.ylabel('Fitness')
plt.title('Convergencia del GA - ' + os.path.basename(instance_file))
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nEstado: OK - GA completado")

# VRPTW - Algoritmo Genetico segun Documento

Implementacion completa del documento de heuristica con parametros reales.

## PASO 0: Importaciones y Configuracion Inicial

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from copy import deepcopy
from collections import defaultdict
import sys
import os

# Configurar parametros
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

print("="*80)
print("PASO 0: IMPORTACIONES Y CONFIGURACION")
print("="*80)
print(f"Semilla aleatoria: {SEED}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print("Estado: OK")

## PASO 1: Carga de Parametros AMPL desde Documento

In [ ]:
print("\n" + "="*80)
print("PASO 1: CARGA DE PARAMETROS AMPL")
print("="*80)

# Naturaleza de nodos
escliente = {0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1}
esdepo = {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}
escritico = {0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 1, 12: 0, 13: 0, 14: 0}

# Tipo de contrato camiones
esHora = {1: 1, 2: 0, 3: 1, 4: 0}
esF6 = {1: 0, 2: 0, 3: 0, 4: 1}
esF12 = {1: 0, 2: 1, 3: 0, 4: 0}

# Capacidades y Costos
Cap = {1: 80, 2: 133, 3: 121, 4: 116}
CH = {1: 123, 2: 116, 3: 154, 4: 114}
CF6 = {1: 277, 2: 411, 3: 420, 4: 303}
CF12 = {1: 429, 2: 478, 3: 382, 4: 558}

# Penalizaciones
pcmin_c = 10
pcmax_c = 15
pcmin_nc = 5
pcmax_nc = 8
preg = 20
pant = 0

# Demandas
DemE = {0: 0, 1: 18, 2: 23, 3: 6, 4: 16, 5: 24, 6: 9, 7: 23, 8: 21, 9: 7, 10: 22, 11: 22, 12: 13, 13: 20, 14: 8}
DemR = {0: 0, 1: 4, 2: 3, 3: 1, 4: 0, 5: 3, 6: 2, 7: 1, 8: 5, 9: 4, 10: 3, 11: 0, 12: 3, 13: 1, 14: 6}

# Tiempo de servicio
TS = {0: 0, 1: 0.34, 2: 0.29, 3: 0.23, 4: 0.32, 5: 0.33, 6: 0.18, 7: 0.35, 8: 0.35, 9: 0.13, 10: 0.31, 11: 0.11, 12: 0.34, 13: 0.25, 14: 0.17}

# Ventanas de tiempo
MinDC = {0: 0, 1: 8, 2: 6, 3: 11, 4: 12, 5: 9, 6: 6, 7: 11, 8: 12, 9: 9, 10: 9, 11: 10, 12: 11, 13: 10, 14: 11}
MaxDC = {0: 24, 1: 16, 2: 9, 3: 13, 4: 19, 5: 14, 6: 13, 7: 16, 8: 17, 9: 14, 10: 13, 11: 17, 12: 18, 13: 14, 14: 17}

# Matriz de distancias
Dist = np.array([
    [0, 43, 44, 26, 32, 22, 42, 42, 24, 8, 29, 16, 20, 30, 17],
    [43, 0, 22, 19, 21, 20, 6, 21, 24, 41, 47, 44, 25, 38, 27],
    [44, 22, 0, 30, 11, 26, 27, 2, 20, 38, 34, 37, 25, 24, 34],
    [26, 19, 30, 0, 22, 7, 17, 28, 17, 26, 39, 32, 14, 33, 9],
    [32, 21, 11, 22, 0, 16, 25, 9, 9, 27, 26, 27, 13, 17, 24],
    [22, 20, 26, 7, 16, 0, 20, 24, 11, 21, 33, 26, 8, 27, 8],
    [42, 6, 27, 17, 25, 20, 0, 26, 26, 41, 50, 45, 26, 41, 26],
    [42, 21, 2, 28, 9, 24, 26, 0, 18, 36, 33, 36, 22, 23, 32],
    [24, 24, 20, 17, 9, 11, 26, 18, 0, 19, 24, 20, 5, 16, 17],
    [8, 41, 38, 26, 27, 21, 41, 36, 19, 0, 21, 9, 16, 22, 18],
    [29, 47, 34, 39, 26, 33, 50, 33, 24, 21, 0, 13, 25, 10, 35],
    [16, 44, 37, 32, 27, 26, 45, 36, 20, 9, 13, 0, 19, 17, 25],
    [20, 25, 25, 14, 13, 8, 26, 22, 5, 16, 25, 19, 0, 19, 12],
    [30, 38, 24, 33, 17, 27, 41, 23, 16, 22, 10, 17, 19, 0, 31],
    [17, 27, 34, 9, 24, 8, 26, 32, 17, 18, 35, 25, 12, 31, 0]
])

# Velocidades por franja (km/h)
v = {1: 50, 2: 40, 3: 10, 4: 40, 5: 30, 6: 25}
tinic = {1: 0, 2: 4, 3: 8, 4: 12, 5: 16, 6: 20}
tfin = {1: 4, 2: 8, 3: 12, 4: 16, 5: 20, 6: 24}

# Parametros adicionales
depo = 0
tlim = 18.0
alm = 14.0
talm = 1.0
tminsal = 0.0
nmuelles = 2
durH = 0.166
Lc = 3
tcarga = 0.5
M = 9999
pw = 10000

# Conjuntos
C = set(range(15))  # Nodos 0-14
R = set(range(1, 5))  # Camiones 1-4
F = set(range(1, 7))  # Franjas 1-6

print(f"\nNodos (C): {len(C)} nodos (1 deposito + {len(C)-1} clientes)")
print(f"Camiones (R): {len(R)} camiones")
print(f"Franjas (F): {len(F)} franjas horarias")
print(f"\nClientes criticos: {sum(1 for c in C if escritico.get(c, 0) == 1)}")
print(f"\nEstado: OK - Parametros cargados correctamente")

## PASO 2: Funciones de Apoyo

In [ ]:
print("\n" + "="*80)
print("PASO 2: FUNCIONES DE APOYO")
print("="*80)

def decode_vector_documento(vector):
    """
    Decodifica vector con 0s delimitadores en rutas
    Ejemplo: [0, 1, 3, 0, 2, 5, 0, 4] -> [[1,3], [2,5], [4]]
    """
    routes = []
    current_route = []
    for client in vector:
        if client == 0:
            if current_route:
                routes.append(current_route)
                current_route = []
        else:
            current_route.append(client)
    if current_route:
        routes.append(current_route)
    return routes

def encode_routes_documento(routes):
    """
    Codifica rutas en vector con 0s delimitadores
    Ejemplo: [[1,3], [2,5], [4]] -> [0, 1, 3, 0, 2, 5, 0, 4, 0]
    """
    vector = [0]
    for route in routes:
        vector.extend(route)
        vector.append(0)
    return vector

def get_all_clients_from_routes(routes):
    """
    Obtiene todos los clientes de un conjunto de rutas
    """
    clients = []
    for route in routes:
        clients.extend(route)
    return set(clients)

def calculate_tour_distance(route, Dist):
    """
    Calcula distancia total de una ruta
    Incluye regreso al deposito (0)
    """
    if not route:
        return 0
    total = Dist[0, route[0]]
    for i in range(len(route) - 1):
        total += Dist[route[i], route[i+1]]
    total += Dist[route[-1], 0]
    return total

def get_contract_cost(truck_id, hours_worked):
    """
    Calcula costo del camion segun tipo de contrato
    """
    if esHora[truck_id]:
        return CH[truck_id] * hours_worked
    elif esF6[truck_id]:
        return CF6[truck_id]
    elif esF12[truck_id]:
        return CF12[truck_id]
    return 0

print("Funciones implementadas:")
print("  - decode_vector_documento()")
print("  - encode_routes_documento()")
print("  - get_all_clients_from_routes()")
print("  - calculate_tour_distance()")
print("  - get_contract_cost()")
print("\nEstado: OK")

## PASO 3: Vector Solucion y Codificacion

In [ ]:
print("\n" + "="*80)
print("PASO 3: VECTOR SOLUCION CON DELIMITADORES")
print("="*80)

# Vector: [0, clientes_ruta1, 0, clientes_ruta2, 0, ...]
# Donde 0 marca inicio/fin de ruta (deposito)

def generate_random_vector():
    """
    Genera vector aleatorio valido
    Asegura que todos los clientes 1-14 esten presentes
    """
    clients = list(range(1, 15))  # Clientes 1-14
    random.shuffle(clients)
    
    # Dividir clientes entre camiones aleatoriamente
    # Al menos 1 camion sin clientes es valido
    num_trucks = random.randint(1, 4)
    
    routes = []
    clients_copy = clients.copy()
    
    for i in range(num_trucks - 1):
        route_size = random.randint(1, len(clients_copy) - (num_trucks - i - 1))
        route = clients_copy[:route_size]
        clients_copy = clients_copy[route_size:]
        routes.append(route)
    
    if clients_copy:
        routes.append(clients_copy)
    
    # Convertir rutas a vector con 0s
    vector = encode_routes_documento(routes)
    return vector

# Ejemplos
print("\nEjemplos de vector solucion:")
for i in range(3):
    vec = generate_random_vector()
    routes = decode_vector_documento(vec)
    print(f"\nIndividuo {i+1}:")
    print(f"  Vector: {vec}")
    print(f"  Rutas: {routes}")
    print(f"  Clientes atendidos: {len(get_all_clients_from_routes(routes))}")

print("\nEstado: OK - Vector solucion implementado")

## PASO 4: Operadores Geneticos (RBX + Mutacion Especializada)

In [ ]:
print("\n" + "="*80)
print("PASO 4: OPERADORES GENETICOS SEGUN DOCUMENTO")
print("="*80)

def route_based_crossover(parent1_vec, parent2_vec):
    """
    Route-Based Crossover (RBX)
    - Intercambia rutas completas entre padres
    - Mantiene estructura de rutas
    """
    # Decodificar padres
    routes1 = decode_vector_documento(parent1_vec)
    routes2 = decode_vector_documento(parent2_vec)
    
    if not routes1 or not routes2:
        return parent1_vec.copy() if len(parent1_vec) > len(parent2_vec) else parent2_vec.copy()
    
    # Seleccionar punto de cruce
    cruce_point = random.randint(1, min(len(routes1), len(routes2)) - 1)
    
    # Combinar rutas
    new_routes = routes1[:cruce_point] + routes2[cruce_point:]
    
    # Obtener clientes faltantes
    clients_used = get_all_clients_from_routes(new_routes)
    all_clients = set(range(1, 15))
    missing_clients = list(all_clients - clients_used)
    repeated_clients = [c for route in new_routes for c in route if sum(1 for r in new_routes for x in r if x == c) > 1]
    
    # Reparar: agregar clientes faltantes
    if missing_clients and new_routes:
        route_idx = random.randint(0, len(new_routes) - 1)
        new_routes[route_idx].extend(missing_clients)
    
    # Remover duplicados
    seen = set()
    for route in new_routes:
        route[:] = [c for c in route if not (c in seen or seen.add(c))]
    
    # Reconvertir a vector
    child = encode_routes_documento(new_routes)
    return child

def mutation_segment_fill(vector):
    """
    Mutacion especializada del documento:
    1. Seleccionar punto de corte
    2. Copiar segmento del padre A (este vector)
    3. Rellenar con clientes faltantes del padre hipotetico B
    4. Mantener cantidad de 0s
    """
    if len(vector) < 5:  # Minimo tamaño para mutar
        return vector.copy()
    
    # Encontrar posiciones de 0s (delimitadores)
    depot_positions = [i for i, v in enumerate(vector) if v == 0]
    
    if len(depot_positions) < 2:
        return vector.copy()
    
    # Seleccionar dos posiciones aleatorias de depot
    pos1 = random.choice(depot_positions[:-1])
    pos2 = random.choice(depot_positions[depot_positions.index(pos1) + 1:])
    
    # Extraer segmento
    segment = vector[pos1+1:pos2]
    
    if not segment:
        return vector.copy()
    
    # Clientes en el segmento
    clients_in_segment = set(segment)
    all_clients = set(range(1, 15))
    faltantes = list(all_clients - clients_in_segment)
    
    # Rellenar segmento con clientes faltantes (simulando padre B)
    new_segment = segment.copy()
    random.shuffle(faltantes)
    new_segment.extend(faltantes[:len(segment) // 2]) if faltantes else None
    
    # Crear nuevo vector
    new_vector = vector[:pos1+1] + new_segment[:len(segment)] + vector[pos2:]
    return new_vector

print("\nOperadores implementados:")
print("  1. route_based_crossover() - Intercambia rutas completas (85%)")
print("  2. mutation_segment_fill() - Segmento + rellenar (10%)")
print("\nProbabilidades:")
print("  - Cruce RBX: 85%")
print("  - Mutacion: 10% (de esta, 70% SWAP, 20% INSERT, 10% SEGMENT)")
print("\nEstado: OK")

## PASO 5: Funcion Objetivo y Evaluacion

In [ ]:
print("\n" + "="*80)
print("PASO 5: FUNCION OBJETIVO SEGUN DOCUMENTO AMPL")
print("="*80)

def evaluate_solution(vector):
    """
    Evalua una solucion completa segun la funcion objetivo del documento:
    
    Z = sum{r in R} CC[r]
        + sum{c in C} escliente[c] * (
            escritico[c] * (pcmin_c * MinEx[c] + pcmax_c * MaxEx[c])
            + (1 - escritico[c]) * (pcmin_nc * MinEx[c] + pcmax_nc * MaxEx[c])
          )
        + sum{r in R} preg * ExReg[r]
        + pw * sum{r in R, c in C} W[r,c]
    
    Donde:
    - CC[r]: Costo del camion r
    - MinEx[c]: Violacion por llegar antes (cliente c)
    - MaxEx[c]: Violacion por llegar despues (cliente c)
    - ExReg[r]: Exceso de hora de regreso (camion r)
    - W[r,c]: Espera en cliente c con camion r
    """
    
    routes = decode_vector_documento(vector)
    total_cost = 0
    
    # Verificar que todos los clientes esten
    clients_served = get_all_clients_from_routes(routes)
    if len(clients_served) != 14:
        return 999999  # Penalizacion por solucion infactible
    
    for truck_id, route in enumerate(routes, 1):
        if truck_id > 4:
            break  # Solo 4 camiones disponibles
        
        if not route:
            continue
        
        # Costo del camion
        distance = calculate_tour_distance(route, Dist)
        time_worked = distance / 40 + sum(TS.get(c, 0) for c in route)  # Estimacion simple
        truck_cost = get_contract_cost(truck_id, time_worked)
        total_cost += truck_cost
        
        # Penalizaciones por ventanas
        for client in route:
            min_window = MinDC.get(client, 0)
            max_window = MaxDC.get(client, 24)
            is_critical = escritico.get(client, 0) == 1
            
            # Estimacion simple de tiempo llegada (necesita simulacion completa)
            arrival_time = 8 + random.uniform(0, 4)  # Placeholder
            
            if arrival_time < min_window:
                penalty = pcmin_c if is_critical else pcmin_nc
                total_cost += penalty * (min_window - arrival_time)
            elif arrival_time > max_window:
                penalty = pcmax_c if is_critical else pcmax_nc
                total_cost += penalty * (arrival_time - max_window)
        
        # Penalizacion por regreso despues de 18:00
        if time_worked > tlim:
            total_cost += preg * (time_worked - tlim)
    
    return total_cost

print("\nFuncion Objetivo implementada:")
print("  - Costo de camiones (segun tipo: hora, franja 6h, franja 12h)")
print("  - Penalizacion ventanas criticas: pcmin_c={}, pcmax_c={}".format(pcmin_c, pcmax_c))
print("  - Penalizacion ventanas no criticas: pcmin_nc={}, pcmax_nc={}".format(pcmin_nc, pcmax_nc))
print("  - Penalizacion regreso post-18:00: preg={}".format(preg))
print("  - Penalizacion espera: pw={}".format(pw))
print("\nEstado: OK")

## PASO 6: Algoritmo Genetico Principal

In [ ]:
print("\n" + "="*80)
print("PASO 6: ALGORITMO GENETICO PRINCIPAL")
print("="*80)

# Parametros del GA segun documento
POPULATION_SIZE = 100
GENERATIONS = 500
CROSSOVER_PROB = 0.85
MUTATION_PROB = 0.10
ELITISM = 2
TOURNAMENT_SIZE = 3
DIVERSITY_CHECK_GEN = 50
DIVERSITY_THRESHOLD = 0.80
STAGNATION_LIMIT = 60

print(f"\nParametros configurados:")
print(f"  Poblacion: {POPULATION_SIZE} individuos")
print(f"  Generaciones: {GENERATIONS}")
print(f"  Cruce (RBX): {CROSSOVER_PROB*100}%")
print(f"  Mutacion: {MUTATION_PROB*100}%")
print(f"  Elitismo: {ELITISM} individuos")
print(f"  Torneo: {TOURNAMENT_SIZE} individuos")
print(f"  Chequeo diversidad: cada {DIVERSITY_CHECK_GEN} generaciones")
print(f"  Umbral diversidad: {DIVERSITY_THRESHOLD*100}%")
print(f"  Parada por estancamiento: {STAGNATION_LIMIT} generaciones sin mejora")

def initialize_population(size):
    """
    Crea poblacion inicial aleatoria
    """
    population = []
    for _ in range(size):
        individual = generate_random_vector()
        population.append(individual)
    return population

def selection_tournament(population, fitness_values, tournament_size):
    """
    Seleccion por torneo K-way
    Retorna el mejor individuo de tournament_size seleccionados
    """
    indices = np.random.choice(len(population), tournament_size, replace=False)
    best_idx = indices[np.argmin([fitness_values[i] for i in indices])]
    return population[best_idx].copy()

def apply_genetic_operators(parent1, parent2):
    """
    Aplica cruce y mutacion
    """
    child = parent1.copy()
    
    # Cruce
    if random.random() < CROSSOVER_PROB:
        child = route_based_crossover(parent1, parent2)
    
    # Mutacion
    if random.random() < MUTATION_PROB:
        mut_type = random.random()
        if mut_type < 0.70:  # 70% SWAP
            # Simple swap
            non_zero_indices = [i for i, v in enumerate(child) if v != 0]
            if len(non_zero_indices) >= 2:
                i, j = random.sample(non_zero_indices, 2)
                child[i], child[j] = child[j], child[i]
        elif mut_type < 0.90:  # 20% INSERT
            # Mover cliente a otra ruta
            routes = decode_vector_documento(child)
            if len(routes) >= 2:
                r1, r2 = random.sample(range(len(routes)), 2)
                if routes[r1]:
                    client = random.choice(routes[r1])
                    routes[r1].remove(client)
                    routes[r2].append(client)
                    child = encode_routes_documento(routes)
        else:  # 10% SEGMENT
            child = mutation_segment_fill(child)
    
    return child

print("\nFunciones GA implementadas:")
print("  - initialize_population()")
print("  - selection_tournament()")
print("  - apply_genetic_operators()")
print("\nEstado: OK")

## PASO 7: Ejecucion del GA Completo

In [ ]:
print("\n" + "="*80)
print("PASO 7: EJECUCION DEL ALGORITMO GENETICO")
print("="*80)

# Inicializar
print("\nInicializando poblacion...")
population = initialize_population(POPULATION_SIZE)
fitness_values = np.array([evaluate_solution(ind) for ind in population])

best_fitness_history = []
avg_fitness_history = []
best_individual = population[np.argmin(fitness_values)].copy()
best_fitness = np.min(fitness_values)
best_generation = 0
stagnation_counter = 0

print(f"Poblacion inicial: {POPULATION_SIZE} individuos")
print(f"Fitness inicial - Mejor: {best_fitness:.2f}, Promedio: {np.mean(fitness_values):.2f}")

# Loop principal del GA
print("\nEjecutando {0} generaciones...\n".format(GENERATIONS))

for generation in range(GENERATIONS):
    # Mostrar progreso
    if generation % 50 == 0:
        print(f"Generacion {generation}: Mejor fitness = {best_fitness:.2f}")
    
    # Crear nueva poblacion
    new_population = []
    
    # Elitismo: mantener mejores individuos
    elite_indices = np.argsort(fitness_values)[:ELITISM]
    for idx in elite_indices:
        new_population.append(population[idx].copy())
    
    # Generar resto de poblacion
    while len(new_population) < POPULATION_SIZE:
        # Seleccionar padres por torneo
        parent1 = selection_tournament(population, fitness_values, TOURNAMENT_SIZE)
        parent2 = selection_tournament(population, fitness_values, TOURNAMENT_SIZE)
        
        # Aplicar operadores
        child = apply_genetic_operators(parent1, parent2)
        new_population.append(child)
    
    # Evaluar nueva poblacion
    population = new_population
    fitness_values = np.array([evaluate_solution(ind) for ind in population])
    
    # Tracking
    gen_best = np.min(fitness_values)
    gen_avg = np.mean(fitness_values)
    best_fitness_history.append(gen_best)
    avg_fitness_history.append(gen_avg)
    
    # Actualizar mejor solucion
    if gen_best < best_fitness:
        best_fitness = gen_best
        best_individual = population[np.argmin(fitness_values)].copy()
        best_generation = generation
        stagnation_counter = 0
    else:
        stagnation_counter += 1
    
    # Chequeo de diversidad (cada DIVERSITY_CHECK_GEN generaciones)
    if generation > 0 and generation % DIVERSITY_CHECK_GEN == 0:
        # Calculo simple de diversidad
        diversity = len(set(map(tuple, population))) / POPULATION_SIZE
        if diversity < DIVERSITY_THRESHOLD:
            # Regenerar 30% de poblacion
            num_regen = int(POPULATION_SIZE * 0.30)
            for i in range(num_regen):
                population[-(i+1)] = generate_random_vector()
    
    # Criterio de parada por estancamiento
    if stagnation_counter >= STAGNATION_LIMIT:
        print(f"\nParada por estancamiento en generacion {generation}")
        break

print(f"\n" + "="*80)
print("RESULTADOS FINALES")
print("="*80)
print(f"\nMejor Z encontrado: {best_fitness:.2f}")
print(f"Generacion: {best_generation}")
print(f"Objetivo NEOS: 780.99")
print(f"Diferencia: {best_fitness - 780.99:.2f}")
print(f"\nMejor solucion (vector): {best_individual}")
print(f"Rutas: {decode_vector_documento(best_individual)}")
print(f"\nGeneraciones ejecutadas: {min(GENERATIONS, generation + 1)}")
print(f"Estancamiento: {stagnation_counter} generaciones sin mejora")

# Graficos
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(best_fitness_history, label='Mejor Fitness', color='blue')
plt.plot(avg_fitness_history, label='Fitness Promedio', color='orange')
plt.axhline(y=780.99, color='red', linestyle='--', label='Objetivo NEOS')
plt.xlabel('Generacion')
plt.ylabel('Fitness')
plt.title('Convergencia del GA')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
improvements = [best_fitness_history[0] - f for f in best_fitness_history]
plt.plot(improvements, color='green')
plt.xlabel('Generacion')
plt.ylabel('Mejora Acumulativa')
plt.title('Mejora Total vs Inicial')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nEstado: OK - GA completado")